In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
directory = '~/PycharmProjects/tfm_hugopobil'

btc_usd_grouped = pd.read_csv(f'{directory}/data/sampled_data/btc_usd_grouped_v2.csv')
tweets_grouped = pd.read_csv(f'{directory}/data/sampled_data/tweets_grouped_v2.csv')

In [3]:
df_clean = pd.read_csv(f'{directory}/data/sampled_data/tweets_clean_v2.csv')
df_clean.head()

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,sample_date
0,21518,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,2021-02-05 10:54:52,perl 0 06 i have insisted that since 0 02 it...,"['bitcoin', 'btc']",Twitter for Android,False,2021-02-05
1,21497,₿ullrun Opinions - Litecoin / BTC/ DOT/ ETH/ UDOO,NaN,"Person with informed opinions about Litecoin, ...",2017-12-11 14:48:20,1553.0,1317.0,9895.0,False,2021-02-05 11:10:09,are we talking about bitcoin sure 17 usd d...,['Bitcoin'],Twitter Web App,False,2021-02-05
2,21469,BittrexPrices,NaN,Scans Bittrex’s main currencies hourly | Also ...,2018-05-25 20:30:07,3130.0,1.0,18.0,False,2021-02-05 11:28:46,prices update in usdt 1 hour btc 376...,NaN,Bittrex Prices,False,2021-02-05
3,21456,mr.tomsquirrle,NaN,New to twitter... Lost my job due to covid 19....,2020-12-30 13:16:53,3.0,41.0,2.0,False,2021-02-05 11:38:33,dominus and johnewbanks i messed up in my firs...,"['dominus', 'johnewbanks', 'pump']",Twitter for iPhone,False,2021-02-05
4,21439,Cryptocurrencies / USD,NaN,Stay updated on the main cryptocurrencies of t...,2018-08-02 10:06:05,3928.0,1.0,10.0,False,2021-02-05 11:46:23,prices update in usd 1 hour btc 37...,NaN,Cryptocurrencies prices to USD,False,2021-02-05


In [4]:
df_clean.dropna(subset=['hashtags'], inplace=True)
df = df_clean[['text']]
df.columns = ['tweets']
df.head()

,tweets
0,perl 0 06 i have insisted that since 0 02 it...
1,are we talking about bitcoin sure 17 usd d...
3,dominus and johnewbanks i messed up in my firs...
6,bitcoin can breakout any second technicala...
7,for cheap network fees coming next month in ...


In [5]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
lem = WordNetLemmatizer()

In [7]:
def cleaning(data):
    tweet_without_url = re.sub(r'http\S+',' ', data)
    tweet_without_hashtag = re.sub(r'#\w+', ' ', tweet_without_url)
    tweet_without_mentions = re.sub(r'@\w+',' ', tweet_without_hashtag)
    precleaned_tweet = re.sub('[^A-Za-z]+', ' ', tweet_without_mentions)

    # Tokenization
    tweet_tokens = TweetTokenizer().tokenize(precleaned_tweet)
    tokens_without_punc = [w for w in tweet_tokens if w.isalpha()]
    tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]

    text_cleaned = [lem.lemmatize(t) for t in tokens_without_sw]

    return " ".join(text_cleaned)

In [8]:
df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
df['date'] = df_clean['date']
df['date_clean'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.drop(columns='date',inplace=True)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_19156/2555527066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_19156/2555527066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df_clean['date']
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_19156/2555527066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,tweets,cleaned_tweets,date_clean
0,perl 0 06 i have insisted that since 0 02 it...,perl insisted since increased month bitcoin btc,2021-02-05
1,are we talking about bitcoin sure 17 usd d...,talking bitcoin sure usd make sense afford u,2021-02-05
3,dominus and johnewbanks i messed up in my firs...,dominus johnewbanks messed first pump please h...,2021-02-05
6,bitcoin can breakout any second technicala...,bitcoin breakout second technicalanalysis intr...,2021-02-05
7,for cheap network fees coming next month in ...,cheap network fee coming next month testnet xo...,2021-02-05


In [9]:
def get_Subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def get_Polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

In [10]:
btc_usd_grouped = btc_usd_grouped.set_index('Date')

In [11]:
btc_usd_grouped

,Adj Close
Date,
2021-02-06,39266.011719
2021-02-07,38903.441406
2021-02-08,46196.464844
2021-02-09,46481.105469
2021-02-10,44918.183594
...,...
2022-01-24,36654.328125
2022-01-25,36954.003906
2022-01-26,36852.121094


In [12]:
def btc_price_cate(score):
    if score < 1:
        return 'negative'
    elif score == 1:
        return 'neutral'
    else:
        return 'positive'


def observe_period(period):
    res = btc_usd_grouped['Adj Close'].shift(period)/btc_usd_grouped['Adj Close']
    res = res.apply(btc_price_cate)
    return res

In [13]:
# Explain in report
# 7 is the days bitcoin price if shifted, to we will predict for week returns
time_sentiment = observe_period(7)
time_sentiment.index

Index(['2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10',
       '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15',
       ...
       '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23',
       '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28'],
      dtype='object', name='Date', length=357)

In [14]:
# CHECK
df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_19156/2522087232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)


In [15]:
# We are going to drop the days for which we cannot measure the crypto_sentiment with the shift
df_2 = df.copy()
df_2 = df_2.dropna()
df_2.shape, df.shape

((20594, 4), (22142, 4))

In [16]:
df_2['subjectivity'] = df_2['cleaned_tweets'].apply(get_Subjectivity)
df_2['polarity'] = df_2['cleaned_tweets'].apply(get_Polarity)
print(df_2.shape)
df_2.head()

(20594, 6)


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity
200,03 00 bitcoin 38 312 0052 bitcoin btc ...,bitcoin bitcoin btc btcusd,2021-02-06,positive,0.000000,0.000000
202,all we need is for bitcoin to break the 3867...,need bitcoin break level go new ath btc btc cr...,2021-02-06,positive,0.454545,0.136364
203,the good news for eth hodlers about insane gas...,good news eth hodlers insane gas fee btc bitco...,2021-02-06,positive,0.800000,-0.150000
204,hbar buy signal pair hbar btc signal p...,hbar buy signal pair hbar btc signal price cro...,2021-02-06,positive,0.000000,0.000000
205,sorry i have to correct you btc 7 figures 20...,sorry correct btc figure next halving bitcoin,2021-02-06,positive,0.500000,-0.250000


In [46]:
def getSentiment(score, threshold=0.5):
    if score < 0:
        return 'negative'
    elif 0.001 < score < threshold:
        return 'neutral'
    else:
        return 'positive'

df_2['sentiment'] = df_2['polarity'].apply(getSentiment)
df_2['target'] = df_2['sentiment'] == df_2['crypto_sentiment']
df_2.head()

,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity,sentiment,target
205,"""Will Institutional Investment Keep Pouring In...",Will Institutional Investment Keep Pouring Int...,2021-02-06,positive,0.0,0.0,positive,True
206,BTC Bitcoin You know where the WSB money is g...,BTC Bitcoin You know WSB money going WallStree...,2021-02-06,positive,0.0,0.0,positive,True
208,"🔼🔼 ₿1 = $38,868 (00:56 UTC)\n$BTC prices conti...",UTC BTC price continue rise Change since midni...,2021-02-06,positive,0.0,0.0,positive,True
209,BTC Bitcoin All the way up! 🚀 🚀 💵 💵 /xVyLbbWRiu,BTC Bitcoin All way xVyLbbWRiu,2021-02-06,positive,0.0,0.0,positive,True
210,Keep going BTC bitcoin,Keep going BTC bitcoin,2021-02-06,positive,0.0,0.0,positive,True


In [49]:
df_2.target.value_counts(normalize=True)

False    0.679115
True     0.320885
Name: target, dtype: float64

In [50]:
df_2.crypto_sentiment.value_counts()

positive    10534
negative    10031
Name: crypto_sentiment, dtype: int64

In [51]:
df_2.sentiment.value_counts(normalize=True)

positive    0.542086
neutral     0.348796
negative    0.109117
Name: sentiment, dtype: float64

In [52]:
df_2 = df_2.set_index('date_clean')
df_2.head()

,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target
date_clean,,,,,,,
2021-02-06,"""Will Institutional Investment Keep Pouring In...",Will Institutional Investment Keep Pouring Int...,positive,0.0,0.0,positive,True
2021-02-06,BTC Bitcoin You know where the WSB money is g...,BTC Bitcoin You know WSB money going WallStree...,positive,0.0,0.0,positive,True
2021-02-06,"🔼🔼 ₿1 = $38,868 (00:56 UTC)\n$BTC prices conti...",UTC BTC price continue rise Change since midni...,positive,0.0,0.0,positive,True
2021-02-06,BTC Bitcoin All the way up! 🚀 🚀 💵 💵 /xVyLbbWRiu,BTC Bitcoin All way xVyLbbWRiu,positive,0.0,0.0,positive,True
2021-02-06,Keep going BTC bitcoin,Keep going BTC bitcoin,positive,0.0,0.0,positive,True


In [53]:
df_2.to_csv(f'{directory}/data/sampled_data/tweets_nlp_modelling_v3.csv')